<div dir="rtl" style="width:90%; margin:auto">
<h3>تعریف Event</h3>
<ol><li><p><strong>Broadcaster (پخش‌کننده)</strong>: کلاسی که Event را تعریف می‌کند و مسئول اعلام Event است. این کلاس حاوی یک delegate است که متدهایی که باید در پاسخ به یک Event فراخوانی شوند را نگه می‌دارد.</p></li><li><p><strong>Subscriber (مشترک)</strong>: کلاسی که به Event علاقه‌مند است و می‌خواهد وقتی Event خاصی رخ دهد، متد خاصی در آن فراخوانی شود. Subscriber می‌تواند با استفاده از عملگرهای += و -= به Event مشترک یا از آن خارج شود.</p></li></ol>
</div>

In [1]:
public delegate void TemperatureChangedEventHandler(object sender, EventArgs e);

public class Thermostat
{
    // تعریف Event بر اساس delegate
    public event TemperatureChangedEventHandler TemperatureChanged;

    // متدی برای بالا بردن دما و فراخوانی Event
    public void SetTemperature(int temperature)
    {
        Console.WriteLine($"Temperature set to {temperature}C");
        OnTemperatureChanged();
    }

    // متد محافظت شده برای فراخوانی delegate
    protected virtual void OnTemperatureChanged()
    {
        TemperatureChanged?.Invoke(this, EventArgs.Empty);
    }
}

public class TemperatureMonitor
{
    public void Subscribe(Thermostat thermostat)
    {
        thermostat.TemperatureChanged += OnTemperatureChanged;
    }

    public void Unsubscribe(Thermostat thermostat)
    {
        thermostat.TemperatureChanged -= OnTemperatureChanged;
    }

    // متدی که در پاسخ به Event فراخوانی می‌شود
    private void OnTemperatureChanged(object sender, EventArgs e)
    {
        Console.WriteLine("Warning: High temperature!");
    }
}


### Standard Event Pattern

<div dir="rtl" style="width:90%; margin:auto">
<h3>1. Delegate Type</h3>
<p>اولین قدم در پیاده‌سازی الگوی استاندارد رویداد، تعریف یک delegate است. Delegate ها در C# به عنوان نوعی اشاره‌گر تابع عمل می‌کنند. در الگوی استاندارد، delegate معمولا دو پارامتر دارد:</p>
<ul><li><strong>sender</strong>: اشاره‌گری به شیء که رویداد را فراخوانی کرده است.</li><li><strong>e</strong>: شیء از نوع <code>EventArgs</code> یا یک کلاس مشتق شده از آن، که حاوی داده‌های مربوط به رویداد است.</li></ul>
<p>معمولا از delegate پیش‌فرض <code>EventHandler</code> یا نسخه‌های ژنریک آن استفاده می‌شود. برای مثال:</p>
</div>

In [ ]:
public delegate void EventHandler(object sender, EventArgs e);
public delegate void EventHandler<TEventArgs>(object sender, TEventArgs e) 
    where TEventArgs : EventArgs;

<div dir="rtl" style="width:90%; margin:auto">
<h3>2. Event Member</h3>
<p>رویداد در کلاسی که "broadcaster" است، تعریف می‌شود. تعریف رویداد با استفاده از کلمه کلیدی <code>event</code> و نوع delegate مربوطه انجام می‌شود. این تعریف به مصرف‌کنندگان کلاس اجازه می‌دهد که به رویداد مشترک (subscribe) شوند.</p>
</div>

In [ ]:
public event EventHandler TemperatureChanged;

<div dir="rtl" style="width:90%; margin:auto">
<h3>3. Method to Raise the Event</h3>
<p>برای فراخوانی رویداد، معمول است که یک متد محافظت‌شده در کلاس تعریف شود که این کار را انجام دهد. این متد معمولا با پیشوند <code>On</code> نام‌گذاری می‌شود و وظیفه‌اش فراخوانی delegate مربوط به رویداد است. این تکنیک اطمینان می‌دهد که رویدادها به صورت صحیح و در زمان مناسب فراخوانی شوند.</p>
</div>

In [ ]:
protected virtual void OnTemperatureChanged(EventArgs e)
{
    TemperatureChanged?.Invoke(this, e);
}


<div dir="rtl" style="width:90%; margin:auto">
<h3>استفاده از EventArgs</h3>
<p>اگر رویداد نیاز به انتقال داده‌های بیشتری دارد، می‌توانید از کلاس‌های مشتق‌شده از <code>EventArgs</code> استفاده کنید تا اطلاعات مورد نیاز را منتقل کنید.</p>
</div>

<div dir="rtl" style="width:90%; margin:auto">
<p>فرض کنید می‌خواهیم یک کلاس به نام <code>Clock</code> تعریف کنیم که هر دقیقه یک رویداد به نام <code>MinutePassed</code> را فراخوانی کند. مشترکان (subscribers) می‌توانند به این رویداد گوش دهند و هر زمان که یک دقیقه گذشت، از آن مطلع شوند. این مثال نه تنها نحوه تعریف یک رویداد و یک delegate را نشان می‌دهد، بلکه چگونگی ارسال داده‌های اضافی را با استفاده از کلاس‌های مشتق‌شده از <code>EventArgs</code> نیز نشان می‌دهد.</p>
</div>

In [1]:
public class MinuteEventArgs : EventArgs
{
    public int Minute { get; }

    public MinuteEventArgs(int minute)
    {
        Minute = minute;
    }
}

public class Clock //broadcaster
{
    public event EventHandler<MinuteEventArgs> MinutePassed;

    protected virtual void OnMinutePassed(MinuteEventArgs e) 
    // حتما باید protected virtual باشد
    //The name must match the name of the event, prefixed with the word On,
    {
        MinutePassed?.Invoke(this, e);
    }

    public void Start()
    {
        for (int minute = 1; minute <= 60; minute++)
        {
            OnMinutePassed(new MinuteEventArgs(minute));
            System.Threading.Thread.Sleep(60000); // فرض بر این است که هر دوره حلقه یک دقیقه طول می‌کشد.
        }
    }
}

public class ClockSubscriber
{
    public void Subscribe(Clock clock)
    {
        clock.MinutePassed += HandleMinutePassed;
    }

    private void HandleMinutePassed(object sender, MinuteEventArgs e)
    {
        Console.WriteLine($"Minute {e.Minute} passed.");
    }

    public void Unsubscribe(Clock clock)
    {
        clock.MinutePassed -= HandleMinutePassed;
    }
}

Clock clock = new Clock();
ClockSubscriber subscriber = new ClockSubscriber();
subscriber.Subscribe(clock);
        
clock.Start();  // شروع به کار ساعت و فراخوانی رویداد هر دقیقه
        
//subscriber.Unsubscribe(clock);  // اختیاری: unsubscribing از رویداد



Minute 1 passed.


Error: Command cancelled.

In [ ]:
public class PriceChangedEventArgs : EventArgs
{
    public readonly decimal LastPrice;
    public readonly decimal NewPrice;
    public PriceChangedEventArgs (decimal lastPrice, decimal newPrice)
    {
        LastPrice = lastPrice; NewPrice = newPrice;
    }
}

public class Stock
{
    string symbol;
    decimal price;
    public Stock (string symbol) => this.symbol = symbol;

    public event EventHandler<PriceChangedEventArgs> PriceChanged;
    protected virtual void OnPriceChanged (PriceChangedEventArgs e)
    {
        PriceChanged?.Invoke (this, e);
    }
    public decimal Price
    {
        get => price;
        set
        {
            if (price == value) return;
            decimal oldPrice = price;
            price = value;
            OnPriceChanged (new PriceChangedEventArgs (oldPrice, price));
        }
    }
}

Stock stock = new Stock ("THPW");
stock.Price = 27.10M;
// Register with the PriceChanged event
stock.PriceChanged += stock_PriceChanged;
stock.Price = 31.59M;

void stock_PriceChanged (object sender, PriceChangedEventArgs e)
{
    if ((e.NewPrice - e.LastPrice) / e.LastPrice > 0.1M)
        Console.WriteLine ("Alert, 10% stock price increase!");
}

### Event Modifiers

In [ ]:
//Like methods, events can be virtual, overridden, abstract, or sealed. 
//Events can also be static:
public class Foo
{
    public static event EventHandler<EventArgs> StaticEvent;
    public virtual event EventHandler<EventArgs> VirtualEvent;
}